In [54]:
from urllib import parse
import pandas as pd
import numpy as np
import lxml
import tensorflow as tf

In [55]:
def l2Norm(inp):# 환율데이터 표본적어서 *10
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp))*10)
    return result

def l3Norm(inp):# *10안함
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp)))
    return result


class Stock:
    
    Exchange='./Exchange.csv' #객체가 공통으로 사용할 환율 데이터 (동적크롤링할줄몰라서 csv로 받아옴)
    
    def __init__(self,code):
        self.code=code
        
    @staticmethod    
    def inputCode():
        k= input("주식 종목코드를 입력하세요: ")
        stock=Stock(str(k))
        print(stock.showChart())
        return 

    @staticmethod
    def exchangeRate():  ##1달간격의 환율의 종가데이터  1행배열 57요소( 18.05.01~  23.01.01) 배열로 출력 SIZE(1,57)
        csv = pd.read_csv(Stock.Exchange,encoding='utf-8')
        #print(csv)  환율 전체데이터 확인하기            
        csv = np.array(csv)
        data=[]
        for i in range(len(csv)):   
            fnum=""
            for j in csv[i][1]:
                if(j == ','):
                    continue
                fnum+=j
            data.append(float(fnum))
        data = np.array(data)
        #정규화
        data = data.astype('float32')
        data = l2Norm(data) 
        return data

    def idxNorm(self):##ROA, ROE, EPS, BPS, DPS, PER, PBR의 각 항목당 (2018.12 ~ 2023.12)의 데이터 6개 갖음 SIZE(7,8)
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        sit=np.array(tables[11]) 
        
        data=[]#정규화 시작
        idx=np.array([])
        for i in range(7):
            idx = sit[17+i][2:8].astype('float32')
            idx =l3Norm(idx)
            data.append(idx)
        data=np.array(data)
        return data

    def showChart(self): ##종목코드의 제무재표를 나타내는 함수
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        df=pd.DataFrame(tables[11])
        display(df)
        return
    

    

In [56]:
#'005930' '000660' '049770' '001570'
Stock.inputCode()

주식 종목코드를 입력하세요: 001570


,IFRS(연결),Annual,Annual.1,Annual.2,Annual.3,Annual.4,Annual.5,Annual.6,Annual.7
0,IFRS(연결),2017/12,2018/12,2019/12,2020/12,2021/12,2022/12(E),2023/12(E),2024/12(E)
1,매출액,1804,2054,1727,1762,2177,NaN,NaN,NaN
2,영업이익,131,135,92,60,121,NaN,NaN,NaN
3,영업이익(발표기준),131,135,92,60,121,NaN,NaN,NaN
4,당기순이익,186,203,-109,-40,133,NaN,NaN,NaN
5,지배주주순이익,187,202,-109,-39,133,NaN,NaN,NaN
6,비지배주주순이익,-0,0,-0,-0,NaN,NaN,NaN,NaN
7,자산총계,1961,2048,2242,2199,3148,NaN,NaN,NaN
8,부채총계,1385,1253,1559,1401,1974,NaN,NaN,NaN
9,자본총계,576,795,683,798,1174,NaN,NaN,NaN


None
